# Importing Dependencies

In [24]:
import pandas as pd

In [93]:
df=pd.read_csv('Deduplication Problem - Sample Dataset.csv')

In [26]:
df.head(8)

,ln,dob,gn,fn
0,SMITH JR,01-03-68,F,WILLIAM
1,ROTHMEYER JR,01-03-68,F,WILLIAM
2,ASBY JR,01-03-68,F,WILLIAM
3,SALTER JR,01-03-68,F,WILLIAM
4,SALTER JR,01-03-68,F,WILLIAM
5,BLAND JR,21-02-62,F,WILLIAM
6,BLAND JR,21-02-62,F,WILLIAM
7,BLAND JR,21-02-62,F,WILLIAM


In [27]:
df.describe()

,ln,dob,gn,fn
count,106,106,106,106
unique,31,23,2,27
top,MICHAELSON JR,07-10-37,M,HAROLD
freq,22,24,80,24


In [28]:
df.nunique()

ln     31
dob    23
gn      2
fn     27
dtype: int64

## Vectorizing the Text & Seperating Dates

In [91]:
from sklearn.feature_extraction.text import CountVectorizer


#from sklearn.feature_extraction.text import TfidfVectorizer


In [30]:
df['day']=pd.to_datetime(df['dob']).dt.day
df['month']=pd.to_datetime(df['dob']).dt.month
df['year']=pd.to_datetime(df['dob']).dt.year

In [31]:
df.drop('dob',axis=1,inplace=True)

In [32]:
gender_dummie=pd.get_dummies(df['gn'])

In [33]:
df=pd.concat([df,gender_dummie],axis=1)
df.drop('gn',axis=1,inplace=True)

In [34]:
from sklearn.cluster import KMeans
K=KMeans()

In [141]:
vectorizer = CountVectorizer()
Vect1=vectorizer.fit_transform(df['fn'])
vectorizer.get_feature_names()
FirstNameCol=pd.DataFrame(Vect1.toarray(),columns=vectorizer.get_feature_names())

vectorizer = CountVectorizer()
vectorizer=CountVectorizer()
Vect2=vectorizer.fit_transform(df['ln'])
SecondNameCol=pd.DataFrame(Vect2.toarray(),columns=vectorizer.get_feature_names())

In [142]:
Final=pd.concat([df,FirstNameCol,SecondNameCol],axis=1)

In [143]:
Final.drop(['ln','fn'],axis=1,inplace=True)


## Creating Manual Feature

In [145]:
from sklearn.metrics.pairwise import cosine_similarity


In [113]:
lenn=[]
similarity_fn=[]
similarity_ln=[]

for i in range(len(df)):
    similarity_fn.append(2*cosine_similarity(Vect1[i], Vect1))
    similarity_ln.append(2*cosine_similarity(Vect2[i], Vect2))
    lenn.append(i)
    

In [140]:
documents = (
    "Dhruv Rawat",
    "Dhruv Raj Rawat",
    "Dhruv Raj Singh Rawat",
    "Dhruv"
    )
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)


array([[1.        , 0.72272912, 0.54347455, 0.63295194]])

In [119]:
similarity_fn_df=pd.DataFrame(similarity_fn.toarray(),columns=lenn)
similarity_fn_df=pd.DataFrame(similarity_fn.toarray(),columns=lenn)

AttributeError: 'list' object has no attribute 'toarray'

In [111]:
similarity_fn_df

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Evaluation

In [38]:
from sklearn.metrics import silhouette_score,silhouette_samples

In [39]:
for i in range(3,100):
    Kmeans_model=KMeans(n_clusters=i).fit(Final)
    labels=Kmeans_model.labels_
    print(i,silhouette_score(Final,labels))

3 0.5068730876428658
4 0.5437471293448615
5 0.5443847603259937
6 0.5568184596644346
7 0.5922436657978688
8 0.6511833959724262
9 0.710474069395153
10 0.7293361822690109
11 0.7543694951576349
12 0.7248176022647028
13 0.7372416876820521
14 0.7450567103358028
15 0.7571205872633404
16 0.762638115290736
17 0.7529578675641283
18 0.7598661950221645
19 0.73691713288784
20 0.747636702299644
21 0.7084640554959896
22 0.7125709811790876
23 0.7037231240490245
24 0.658479888351564
25 0.6609455041910807
26 0.6997311082937872
27 0.649303379589222
28 0.6821558354428349
29 0.6779223061130436
30 0.6769936329723707
31 0.6895805871017627
32 0.716787614284042
33 0.6926248657794108
34 0.6893813549244217
35 0.6759986986119794
36 0.6709357287248313
37 0.6549168286019597
38 0.6887900220785186
39 0.7037282165151095
40 0.656166256419702
41 0.6830529785877464
42 0.6774886028447676
43 0.6656043658405258
44 0.6810591058220908
45 0.6689505907321349
46 0.6361269836933148
47 0.6506856440746693
48 0.6540063916164657
49 0

In [60]:
Kmeans_model=KMeans(n_clusters=60).fit(Final)
labels=Kmeans_model.labels_

In [61]:
labels

array([57, 38, 58,  1,  1, 28, 28, 28, 35, 35, 59, 36,  7, 28, 11, 11, 41,
       41, 21, 44, 48, 33,  2, 19, 43, 19, 43, 15, 12, 12, 42, 18, 46, 39,
       20, 27, 27, 27,  6,  6, 31, 31, 55, 14, 23, 49,  3,  8, 40, 40, 25,
       25, 25, 24, 24, 34, 34, 34,  4,  4, 51, 13, 29, 29, 47, 54, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 26, 26, 26, 26, 37, 37, 52, 26, 26, 30,
       30, 45, 32, 22, 56, 16, 16, 16, 53,  9,  9,  9,  9,  9, 17,  0,  0,
        0, 50,  5,  5])

In [64]:
Score=pd.DataFrame(labels,columns=['Similarity'])

In [66]:
pd.concat([df,Score],axis=1).to_csv("clustered.csv",index=False)
